In [1]:
import numpy as np

In [2]:
from uflp_utils import read_bk_instance, objective

In [3]:
user_cost, resource_cost = read_bk_instance('BildeKrarup/B/B1.1')
user_cost, resource_cost

(array([[707, 100, 395, ..., 770, 212, 731],
        [164, 584, 976, ..., 171, 301, 413],
        [ 75, 300,  93, ..., 473, 647,  50],
        ...,
        [139,  65, 745, ...,   0, 377, 715],
        [626, 825, 161, ..., 965, 682, 858],
        [369, 270, 985, ..., 189, 434, 201]], shape=(100, 50)),
 array([4751, 3011, 7433, 3947, 8799, 9776, 8951, 9182, 7895, 9103, 1736,
        3485, 9627, 2090, 2501, 3527, 5234, 8975, 3751, 6554, 9543, 7599,
        6022, 4293, 6957, 1866, 7764, 8018, 4583, 3907, 7729, 1244, 2079,
        6172, 7198, 2024, 7351, 6174, 3889, 7315, 6641, 7092, 9907, 1373,
        9155, 2961, 4548, 7670, 2782, 6664]))

In [4]:
class Individual:
    def __init__(self, user_cost: np.ndarray, resource_cost: np.ndarray):
        self.code = np.random.randint(0, 2, size=len(resource_cost))

        self.user_cost = user_cost
        self.resource_cost = resource_cost
        
        self.fitness = self.calc_fitness()

    def calc_fitness(self):
        return -objective(self.code, self.user_cost, self.resource_cost)
    
    def __lt__(self, other):
        return self.fitness < other.fitness

In [ ]:
from copy import deepcopy
import random


class GeneticAlgorithm:
    def __init__(
            self,
            population_size: int,
            num_generations: int,
            mutation_prob: float,
            elitism_size: float,
            crossover_type: str,
            selection_type: str,
            tournament_size: int | None,
        ):
        self.population_size = population_size
        self.num_elite = int(population_size * elitism_size)
        if self.num_elite % 2 != self.population_size % 2:
            self.num_elite += 1
        self.mutation_prob = mutation_prob
        self.selection_type = selection_type
        if selection_type == 'tournament':
            assert tournament_size is not None
        self.tournament_size = tournament_size
        self.num_generations = num_generations
        self.crossover_type = crossover_type

    def selection(self, population: list[Individual]):
        match self.selection_type:
            case 'tournament':
                participants = random.sample(population, self.tournament_size)
                return max(participants)
            case 'roulette':
                # za domaci
                pass
            case 'rang':
                # za domaci
                pass
            case _:
                raise ValueError(f'unknown selection_type: {self.selection_type}')

    def crossover(self, parent1_code: np.array, parent2_code: np.array) -> tuple[np.array, np.array]:
        match self.crossover_type:
            case 'single_point':
                breakpoint = random.randrange(1, len(parent1_code) - 1)

                child1_code = np.zeros_like(parent1_code)
                child2_code = np.zeros_like(parent2_code)
                
                child1_code[:breakpoint] = parent1_code[:breakpoint]
                child1_code[breakpoint:] = parent2_code[breakpoint:]

                child2_code[:breakpoint] = parent2_code[:breakpoint]
                child2_code[breakpoint:] = parent1_code[breakpoint:]

                return child1_code, child2_code
            case 'uniform':
                # za domaci
                pass
            case _:
                raise ValueError(f'unknown crossover_type: {self.crossover_type}')

    def mutation(self, ind_code: np.array) -> np.array:
        res_code = np.zeros_like(ind_code)
        for i in range(len(ind_code)):
            if random.random() < self.mutation_prob:
                res_code[i] = 1 - ind_code[i]
            else:
                res_code[i] = ind_code[i]
        return res_code

    def solve(self, user_cost: np.ndarray, resource_cost: np.ndarray) -> Individual:
        population = [Individual(user_cost, resource_cost) for _ in range(self.population_size)]
        for _ in range(self.num_generations):
            population.sort(reverse=True)
            new_population = population[:self.num_elite]
            
            for i in range(self.num_elite, self.population_size, 2):
                parent1 = self.selection(population)
                parent2 = self.selection(population)

                child1_code, child2_code = self.crossover(parent1.code, parent2.code)

                child1_code = self.mutation(child1_code)
                child2_code = self.mutation(child2_code)

                child1 = Individual(user_cost, resource_cost)
                child1.code = child1_code
                child1.fitness = child1.calc_fitness()
                child2 = Individual(user_cost, resource_cost)
                child2.code = child2_code
                child2.fitness = child2.calc_fitness()
                
                new_population.append(child1)
                new_population.append(child2)

            population = deepcopy(new_population)

        return max(population)

In [6]:
ga = GeneticAlgorithm(population_size=100, num_generations=100, mutation_prob=0.05, elitism_size=0.07, selection_type='tournament', tournament_size=10, crossover_type='single_point')

In [9]:
res = ga.solve(user_cost, resource_cost)

In [10]:
res.fitness

np.int64(-23468)